In [1]:
!pip install js2xml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.0/292.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.3 MB/s eta 0:00:00


In [2]:
import requests
import re
from bs4 import BeautifulSoup
import js2xml
import json
import pandas as pd
from datetime import date, time, datetime

## Scraping Covid data

In [3]:
response = requests.get('https://www.worldometers.info/coronavirus/country/india/')

In [4]:
soup = BeautifulSoup(response.content)

In [5]:
covid_data_raw = soup.find_all('script',  string=re.compile("Highcharts.chart"))

In [6]:

def extract_dates(covid_data_raw):
    temp = re.findall('categories: \[[^\]]*\]',str(covid_data_raw[0].string))
    temp = re.sub('"|,', '', temp[0])
    return re.findall('[a-zA-Z]{3} [0-9]{2} [0-9]{4}', temp)

In [7]:


data = []
columns = []

for i in covid_data_raw:
    data_string = str(i.string)
    for j in re.findall('data: \[[^\]]*\]', data_string):
        data.append(j[7:-1].split(','))
    for j in re.findall('name: [^,]*,', data_string):
        columns.append(re.findall(" '.*',",j)[0].replace("'", "").replace(",", "").strip())

In [8]:
data= [list(x) for x in zip(*data)]

In [9]:
covid_data = pd.DataFrame(data, columns= columns, index=extract_dates(covid_data_raw))
covid_data

,Death Rate,Recovery Rate,Cases,Cases,Daily Cases,3-day moving average,7-day moving average,Currently Infected,Deaths,Deaths,Daily Deaths,3-day moving average,7-day moving average,New Recoveries,New Cases,Death Rate,Recovery Rate
Feb 15 2020,0,100,3,3,null,null,null,0,0,0,null,null,null,null,null,0,100
Feb 16 2020,0,100,3,3,0,0,0,0,0,0,null,null,null,0,0,0,100
Feb 17 2020,0,100,3,3,0,0,0,0,0,0,null,null,null,0,0,0,100
Feb 18 2020,0,100,3,3,0,0,0,0,0,0,null,null,null,0,0,0,100
Feb 19 2020,0,100,3,3,0,0,0,0,0,0,null,null,null,0,0,0,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Apr 14 2023,1.18999999999999994670929481799248605966567993...,98.81000000000000227373675443232059478759765625,44808022,44808022,10753,10673,8109,53720,531091,531091,27,25,19,6628,10753,1.18999999999999994670929481799248605966567993...,98.81000000000000227373675443232059478759765625
Apr 15 2023,1.18999999999999994670929481799248605966567993...,98.81000000000000227373675443232059478759765625,44818115,44818115,10093,10652,8786,57542,531114,531114,23,27,22,6248,10093,1.18999999999999994670929481799248605966567993...,98.81000000000000227373675443232059478759765625
Apr 16 2023,1.18999999999999994670929481799248605966567993...,98.81000000000000227373675443232059478759765625,44827226,44827226,9111,9986,9247,60313,531141,531141,27,25,23,6313,9111,1.18999999999999994670929481799248605966567993...,98.81000000000000227373675443232059478759765625
Apr 17 2023,1.18999999999999994670929481799248605966567993...,98.81000000000000227373675443232059478759765625,44834859,44834859,7633,8945,9527,61233,531152,531152,11,21,22,6702,7633,1.18999999999999994670929481799248605966567993...,98.81000000000000227373675443232059478759765625


## Gold price scraping

In [57]:
response = requests.get('https://in.investing.com/commodities/gold-historical-data?end_date=1681916079&st_date=1579458600&interval_sec=weekly&interval_sec=daily')

In [58]:
soup = BeautifulSoup(response.content)